In [147]:
import pandas as pd
import pandas_ta as ta
import importlib
import os
import sys
import yfinance as yf
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

Input needed:

1. prices.csv
-date, asset, Close, Expected Return (from AR model), other components needed to run strategies

2. strategy_map.csv
- asset, strategy (e.g., Strategy05), ideal_proportion

Config Example:

config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "cash_ceiling_pct": 0.30,      # If cash > 30%, consider buying gold
    "fee": 0.001,                  # Transaction fee (0.1%)
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

Notes: the tp, sl pct gonna make "dynamic" - different for each asset.

In [171]:
# Create some sample data to try now 
config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

filename_to_classname = {
    'larry_williams_price_action': 'LarryWilliamsPriceAction',
    'macd_bollinger_bands_mean_reversion': 'MACDBollingerBandsMeanReversion',
    'michael_harris_price_action' : 'MichaelHarrisPriceAction',
    'rsi_divergence': 'RSIDivergence',
    'scalping': 'Scalping',
    'volume_spike_reversal': 'VolumeSpikeReversal', 
    'mean_reversion': 'MeanReversion'
}

strategy_map = pd.read_csv("../data/processed/asset_strategies_copy.csv").set_index("asset")
strategy_map

,weight,strategy,return,type
asset,,,,
CZR,0.195536,Scalping,4.840157,Stock
INTC,0.159487,Mean_Reversion,29.333192,Stock
MHK,0.115818,Mean_Reversion,14.363444,Stock
BLDR,0.107738,RSI_Divergence,25.424814,Stock
URI,0.088408,Michael_Harris_Price_Action,15.062324,Stock
ON,0.070599,RSI_Divergence,36.768206,Stock
NCLH,0.031879,Larry_Williams_Price_Action,64.092653,Stock
ALB,0.028309,RSI_Divergence,41.214243,Stock
VST,0.005340,Larry_Williams_Price_Action,113.479470,Stock


First function retrieves data from yf 

In [149]:
def get_asset_df(strategy_map, start, end):
    df_list = []
    for asset_name in strategy_map.index:
        # Retrieve yf finance data 
        df = yf.Ticker(asset_name).history(start=start, end=end, actions=False)

        # Align date formatting 
        df = df.reset_index()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

        df['Asset'] = asset_name
        df_list.append(df.reset_index())

    df_all = pd.concat(df_list).set_index(['Date', 'Asset']).sort_index()
    return df_all 

In [150]:
asset_df = get_asset_df(strategy_map=strategy_map, start= "2024-01-01", end="2025-03-01")

asset_df.head(10)

index        Open        High         Low       Close  \
Date       Asset                                                             
2024-01-01 AVAX-USD      0   38.558670   41.980568   38.171398   41.864464   
           LINK-USD      0   14.946928   15.559930   14.874288   15.535809   
           SOL-USD       0  101.512810  109.508682  101.512810  109.508682   
2024-01-02 ALB           0  140.026594  146.499153  138.322769  143.835709   
           AVAX-USD      1   41.869736   43.391705   40.468700   40.645210   
           BLDR          0  163.869995  167.089996  161.899994  165.380005   
           CZR           0   46.560001   48.570000   46.169998   47.779999   
           INTC          0   48.549777   48.727398   46.822905   47.168278   
           LINK-USD      1   15.537648   16.046949   15.118124   15.165293   
           MHK           0  102.019997  105.500000  101.010002  105.279999   

                         Volume  
Date       Asset                 
2024-01-01 AVAX-USD   671490413  
           LINK-USD   319817917  
           SOL-USD   2157671990  
2024-01-02 ALB          1962300  
           AVAX-USD   869881311  
           BLDR         1320300  
           CZR          2807800  
           INTC        45905700  
           LINK-USD   500000291  
           MHK           800200

Second function implements AR model to derive expected return 

In [151]:
def compute_returns(asset_df):
    asset_df["Return"] = asset_df.groupby("Asset")["Close"].pct_change()
    return asset_df

asset_df_with_returns = compute_returns(asset_df)

In [152]:
asset_df_with_returns.head(10)

index        Open        High         Low       Close  \
Date       Asset                                                             
2024-01-01 AVAX-USD      0   38.558670   41.980568   38.171398   41.864464   
           LINK-USD      0   14.946928   15.559930   14.874288   15.535809   
           SOL-USD       0  101.512810  109.508682  101.512810  109.508682   
2024-01-02 ALB           0  140.026594  146.499153  138.322769  143.835709   
           AVAX-USD      1   41.869736   43.391705   40.468700   40.645210   
           BLDR          0  163.869995  167.089996  161.899994  165.380005   
           CZR           0   46.560001   48.570000   46.169998   47.779999   
           INTC          0   48.549777   48.727398   46.822905   47.168278   
           LINK-USD      1   15.537648   16.046949   15.118124   15.165293   
           MHK           0  102.019997  105.500000  101.010002  105.279999   

                         Volume    Return  
Date       Asset                           
2024-01-01 AVAX-USD   671490413       NaN  
           LINK-USD   319817917       NaN  
           SOL-USD   2157671990       NaN  
2024-01-02 ALB          1962300       NaN  
           AVAX-USD   869881311 -0.029124  
           BLDR         1320300       NaN  
           CZR          2807800       NaN  
           INTC        45905700       NaN  
           LINK-USD   500000291 -0.023849  
           MHK           800200       NaN

In [153]:
def generate_expected_returns(data, ar_results):
    """ Calculate expected returns using fitted AR models """
    new_data = data.reset_index().merge(ar_results, on="Asset", how="left").set_index(["Date", "Asset"])
    new_data["Expected Return"] = new_data["const"] + new_data["Return"] * new_data["ar.L1"]
    return new_data

In [154]:
ar_model_info = pd.read_csv("../data/processed/ar_model.csv")
ar_model_info

,Asset,Best Model,BIC,const,ar.L1,sigma2
0,CZR,"(1, 0, 0)",-2137.099847,-0.001004,-0.037306,0.000487
1,INTC,"(1, 0, 0)",-1898.767394,-0.001165,0.053588,0.000827
2,MHK,"(1, 0, 0)",-2237.419365,0.000454,0.064773,0.000389
3,BLDR,"(1, 0, 0)",-2146.278473,-0.000304,-0.037379,0.000477
4,URI,"(1, 0, 0)",-2284.821016,0.000504,-0.071571,0.000350
5,ON,"(1, 0, 0)",-2053.295520,-0.001143,-0.153363,0.000586
6,NCLH,"(1, 0, 0)",-2008.360786,0.000543,-0.014170,0.000648
7,ALB,"(1, 0, 0)",-1908.997544,-0.000997,-0.151407,0.000808
8,VST,"(1, 0, 0)",-1739.133780,0.003166,-0.093050,0.001179
9,AVAX-USD,"(1, 0, 0)",-1402.471749,-0.000203,0.003612,0.002473


In [155]:
# Apply AR model
asset_df_with_AR = generate_expected_returns(asset_df_with_returns, ar_model_info)
asset_df_with_AR = asset_df_with_AR.drop(['Best Model','BIC', 'const','ar.L1', 'sigma2'], axis=1)
asset_df_with_AR

index        Open        High         Low       Close  \
Date       Asset                                                             
2024-01-01 AVAX-USD      0   38.558670   41.980568   38.171398   41.864464   
           LINK-USD      0   14.946928   15.559930   14.874288   15.535809   
           SOL-USD       0  101.512810  109.508682  101.512810  109.508682   
2024-01-02 ALB           0  140.026594  146.499153  138.322769  143.835709   
           AVAX-USD      1   41.869736   43.391705   40.468700   40.645210   
...                    ...         ...         ...         ...         ...   
2025-02-28 NCLH        290   23.540001   23.680000   22.280001   22.719999   
           ON          290   47.939999   48.520000   46.040001   47.049999   
           SOL-USD     424  137.620773  148.183838  125.742218  148.030014   
           URI         290  635.500000  643.719971  627.760010  642.320007   
           VST         290  127.779188  134.477620  123.836006  133.429428   

                         Volume    Return  Expected Return  
Date       Asset                                            
2024-01-01 AVAX-USD   671490413       NaN              NaN  
           LINK-USD   319817917       NaN              NaN  
           SOL-USD   2157671990       NaN              NaN  
2024-01-02 ALB          1962300       NaN              NaN  
           AVAX-USD   869881311 -0.029124        -0.000308  
...                         ...       ...              ...  
2025-02-28 NCLH        24852400 -0.042159         0.001140  
           ON          13110600 -0.006965        -0.000075  
           SOL-USD   7544673033  0.075637        -0.004608  
           URI           694800  0.014451        -0.000530  
           VST         13060900  0.028075         0.000554  

[3894 rows x 8 columns]

In [156]:
def calculate_technical_indicators(df):
    df = df.copy()

    # Core indicators
    df["EMA_50"] = ta.ema(df["Close"], length=50)
    df["EMA_200"] = ta.ema(df["Close"], length=200)
    df["RSI"] = ta.rsi(df["Close"], length=14)
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=7)

    # Bollinger Bands
    bbands = ta.bbands(df["Close"], length=20)
    bbands = bbands.rename(columns={
        "BBU_20_2.0": "Upper_Band",
        "BBM_20_2.0": "Middle_Band",
        "BBL_20_2.0": "Lower_Band",
        "BBB_20_2.0": "Band_Width",
        "BBP_20_2.0": "Percent_B",
    })

    # Stochastic Oscillator
    stoch = ta.stoch(df["High"], df["Low"], df["Close"])
    stoch = stoch.rename(columns={
        "STOCHk_14_3_3": "%K",
        "STOCHd_14_3_3": "%D"
    })

    # MACD
    macd = ta.macd(df["Close"])
    macd = macd.rename(columns={
        "MACD_12_26_9": "MACD",
        "MACDh_12_26_9": "Histogram",
        "MACDs_12_26_9": "Signal"
    })

    # Join indicator DataFrames
    df = df.join([bbands, stoch, macd])

    # On-Balance Volume
    df["OBV"] = ta.obv(df["Close"], df["Volume"])

    return df

In [157]:
enriched_df = calculate_technical_indicators(asset_df_with_AR)
enriched_df

index        Open        High         Low       Close  \
Date       Asset                                                             
2024-01-01 AVAX-USD      0   38.558670   41.980568   38.171398   41.864464   
           LINK-USD      0   14.946928   15.559930   14.874288   15.535809   
           SOL-USD       0  101.512810  109.508682  101.512810  109.508682   
2024-01-02 ALB           0  140.026594  146.499153  138.322769  143.835709   
           AVAX-USD      1   41.869736   43.391705   40.468700   40.645210   
...                    ...         ...         ...         ...         ...   
2025-02-28 NCLH        290   23.540001   23.680000   22.280001   22.719999   
           ON          290   47.939999   48.520000   46.040001   47.049999   
           SOL-USD     424  137.620773  148.183838  125.742218  148.030014   
           URI         290  635.500000  643.719971  627.760010  642.320007   
           VST         290  127.779188  134.477620  123.836006  133.429428   

                         Volume    Return  Expected Return      EMA_50  \
Date       Asset                                                         
2024-01-01 AVAX-USD   671490413       NaN              NaN         NaN   
           LINK-USD   319817917       NaN              NaN         NaN   
           SOL-USD   2157671990       NaN              NaN         NaN   
2024-01-02 ALB          1962300       NaN              NaN         NaN   
           AVAX-USD   869881311 -0.029124        -0.000308         NaN   
...                         ...       ...              ...         ...   
2025-02-28 NCLH        24852400 -0.042159         0.001140  108.488325   
           ON          13110600 -0.006965        -0.000075  106.078979   
           SOL-USD   7544673033  0.075637        -0.004608  107.724118   
           URI           694800  0.014451        -0.000530  128.688662   
           VST         13060900  0.028075         0.000554  128.874575   

                        EMA_200  ...  Middle_Band  Upper_Band  Band_Width  \
Date       Asset                 ...                                        
2024-01-01 AVAX-USD         NaN  ...          NaN         NaN         NaN   
           LINK-USD         NaN  ...          NaN         NaN         NaN   
           SOL-USD          NaN  ...          NaN         NaN         NaN   
2024-01-02 ALB              NaN  ...          NaN         NaN         NaN   
           AVAX-USD         NaN  ...          NaN         NaN         NaN   
...                         ...  ...          ...         ...         ...   
2025-02-28 NCLH      120.707891  ...    92.489464  357.740501  573.581084   
           ON        119.974976  ...    90.915552  356.851953  585.018505   
           SOL-USD   120.254131  ...    97.192399  362.297396  545.526193   
           URI       125.448816  ...   122.350900  478.482326  582.147622   
           VST       125.528225  ...   127.359871  481.147924  555.572254   

                     Percent_B         %K         %D       MACD  Histogram  \
Date       Asset                                                             
2024-01-01 AVAX-USD        NaN        NaN        NaN        NaN        NaN   
           LINK-USD        NaN        NaN        NaN        NaN        NaN   
           SOL-USD         NaN        NaN        NaN        NaN        NaN   
2024-01-02 ALB             NaN        NaN        NaN        NaN        NaN   
           AVAX-USD        NaN        NaN        NaN        NaN        NaN   
...                        ...        ...        ...        ...        ...   
2025-02-28 NCLH       0.368484   5.951138   4.459002 -17.470936 -10.763449   
           ON         0.417526   7.623598   6.503358 -19.031284  -9.859038   
           SOL-USD    0.595882   9.203011   7.592582 -11.981513  -2.247413   
           URI        1.230024  41.351621  19.392743  33.108894  34.274395   
           VST        0.508578  45.833511  32.129381  27.463567  22.903254   

                       Sig

In [158]:
updated_df = enriched_df.loc[enriched_df.index.get_level_values('Date') >= '2025-01-01']
updated_df

index        Open        High         Low       Close  \
Date       Asset                                                             
2025-01-01 AVAX-USD    366   35.690929   37.869911   34.960541   37.693439   
           LINK-USD    366   20.001163   21.771200   19.707378   21.674845   
           SOL-USD     366  189.266922  194.818497  187.879456  193.873734   
2025-01-02 ALB         252   85.956465   87.497790   83.937822   84.753235   
           AVAX-USD    367   37.693447   40.501595   37.689964   39.229160   
...                    ...         ...         ...         ...         ...   
2025-02-28 NCLH        290   23.540001   23.680000   22.280001   22.719999   
           ON          290   47.939999   48.520000   46.040001   47.049999   
           SOL-USD     424  137.620773  148.183838  125.742218  148.030014   
           URI         290  635.500000  643.719971  627.760010  642.320007   
           VST         290  127.779188  134.477620  123.836006  133.429428   

                         Volume    Return  Expected Return      EMA_50  \
Date       Asset                                                         
2025-01-01 AVAX-USD   334807162  0.056107    -6.013646e-07  134.809535   
           LINK-USD   670017230  0.083668     1.535036e-03  130.372880   
           SOL-USD   2324231668  0.024354    -5.260706e-04  132.863110   
2025-01-02 ALB          1866100 -0.009875     4.978192e-04  130.976448   
           AVAX-USD   522067297  0.040742    -5.610548e-05  127.378515   
...                         ...       ...              ...         ...   
2025-02-28 NCLH        24852400 -0.042159     1.140214e-03  108.488325   
           ON          13110600 -0.006965    -7.505876e-05  106.078979   
           SOL-USD   7544673033  0.075637    -4.607648e-03  107.724118   
           URI           694800  0.014451    -5.301611e-04  128.688662   
           VST         13060900  0.028075     5.537452e-04  128.874575   

                        EMA_200  ...  Middle_Band  Upper_Band  Band_Width  \
Date       Asset                 ...                                        
2025-01-01 AVAX-USD  134.551577  ...   143.823889  533.029612  541.225421   
           LINK-USD  133.428425  ...   143.878457  533.015349  540.924472   
           SOL-USD   134.029871  ...   147.645144  537.186242  527.672077   
2025-01-02 ALB       133.539556  ...   150.595306  537.282319  513.544579   
           AVAX-USD  132.601144  ...   149.383764  537.305079  519.362086   
...                         ...  ...          ...         ...         ...   
2025-02-28 NCLH      120.707891  ...    92.489464  357.740501  573.581084   
           ON        119.974976  ...    90.915552  356.851953  585.018505   
           SOL-USD   120.254131  ...    97.192399  362.297396  545.526193   
           URI       125.448816  ...   122.350900  478.482326  582.147622   
           VST       125.528225  ...   127.359871  481.147924  555.572254   

                     Percent_B         %K         %D       MACD  Histogram  \
Date       Asset                                                             
2025-01-01 AVAX-USD   0.363658  39.559111  43.243440  18.767103   8.649790   
           LINK-USD   0.342981   6.649190  31.031936   6.163310  -3.163203   
           SOL-USD    0.559337   9.372749  18.527017   9.954970   0.502766   
2025-01-02 ALB        0.414864  11.653695   9.225211   4.107428  -4.275821   
           AVAX-USD   0.358019  12.501912  11.176119  -4.152338 -10.028470   
...                        ...        ...        ...        ...        ...   
2025-02-28 NCLH       0.368484   5.951138   4.459002 -17.470936 -10.763449   
           ON         0.417526   7.623598   6.503358 -19.031284  -9.859038   
           SOL-USD    0.595882   9.203011   7.592582 -11.981513  -2.247413   
           URI        1.230024  41.351621  19.392743  33.108894  34.274395   
           VST        0.508578  45.833511  32.129381  27.463567  22.903254   

                        Si

Third function to add signal from strategies 

In [134]:
#previous load data and apply strategies function -> does not work as strategies files have changed 

def load_data_and_apply_strategies(portfolio, strategy_map):
    """
    Function takes in portfolio data (for a date range) and generates signals for HOLD, LONG, SHORT 
    """

    df_list = []
    for asset_name in strategy_map.index:
        strat_file_name = strategy_map.loc[asset_name, "strategy"].lower()
        strat_class_name = filename_to_classname[strat_file_name]
        
        # Load strategy class dynamically
        strategy_module = importlib.import_module(f'strategies.{strat_file_name}')
        strategy_class = getattr(strategy_module, strat_class_name)

        # Apply strategy to class 
        #asset = portfolio.get_asset(asset_name)
        asset_df_with_signals = asset.apply_strategy(strategy_class)

        asset_df_with_signals['Asset'] = asset_name
        df_list.append(asset_df_with_signals.reset_index())
    
    df_with_signals = pd.concat(df_list).set_index(['Date', 'Asset']).sort_index()
    
    return df_with_signals

In [159]:
def load_data_and_apply_strategies(df_all, strategy_map):
    df_list = []

    for asset_name in strategy_map.index:
        try:
            # Get strategy module + class
            strat_file_name = strategy_map.loc[asset_name, "strategy"].lower()
            strat_class_name = filename_to_classname[strat_file_name]
            
            strategy_module = importlib.import_module(f"strategies.custom.{strat_file_name}")
            strategy_class = getattr(strategy_module, strat_class_name)
            strategy_instance = strategy_class()

            # Get asset's DataFrame slice
            asset_df = df_all.xs(asset_name, level="Asset").copy()

            # Apply strategy
            asset_df = strategy_instance.generate_signals(asset_df)

            # Restore asset label if removed
            asset_df["Asset"] = asset_name
            df_list.append(asset_df.reset_index())

        except Exception as e:
            print(f"Error processing {asset_name} ({strat_class_name}): {e}")
            continue

    # Combine and restore multi-index
    df_with_signals = pd.concat(df_list).set_index(['Date', 'Asset']).sort_index()
    return df_with_signals


In [160]:
df_with_signals = load_data_and_apply_strategies(updated_df, strategy_map)

Error processing AVAX-USD (MACDBollingerBandsMeanReversion): 'NoneType' object is not subscriptable
Error processing SOL-USD (VolumeSpikeReversal): 'EMA_50'


In [162]:
df_with_signals

level_0  index        Open        High         Low  \
Date       Asset                                                          
2025-01-01 LINK-USD      NaN    366   20.001163   21.771200   19.707378   
2025-01-02 ALB           NaN    252   85.956465   87.497790   83.937822   
           BLDR          NaN    252  145.089996  146.029999  141.149994   
           CZR           0.0    252   33.660000   34.084999   32.349998   
           INTC          NaN    252   20.230000   20.430000   20.000000   
...                      ...    ...         ...         ...         ...   
2025-02-28 MHK           NaN    290  117.080002  118.199997  116.239998   
           NCLH          NaN    290   23.540001   23.680000   22.280001   
           ON            NaN    290   47.939999   48.520000   46.040001   
           URI           NaN    290  635.500000  643.719971  627.760010   
           VST           NaN    290  127.779188  134.477620  123.836006   

                          Close     Volume    Return  Expected Return  \
Date       Asset                                                        
2025-01-01 LINK-USD   21.674845  670017230  0.083668         0.001535   
2025-01-02 ALB        84.753235    1866100 -0.009875         0.000498   
           BLDR      141.979996     906800 -0.006647        -0.000056   
           CZR        32.590000    4674200 -0.024835        -0.000078   
           INTC       20.219999   46438500  0.008479        -0.000710   
...                         ...        ...       ...              ...   
2025-02-28 MHK       117.589996     709000  0.009703         0.001083   
           NCLH       22.719999   24852400 -0.042159         0.001140   
           ON         47.049999   13110600 -0.006965        -0.000075   
           URI       642.320007     694800  0.014451        -0.000530   
           VST       133.429428   13060900  0.028075         0.000554   

                         EMA_50  ...  Band_Width  Percent_B         %K  \
Date       Asset                 ...                                     
2025-01-01 LINK-USD  130.372880  ...  540.924472   0.342981   6.649190   
2025-01-02 ALB       130.976448  ...  513.544579   0.414864  11.653695   
           BLDR      127.951122  ...  527.398775   0.493610   9.994429   
           CZR       124.211470  ...  517.732892   0.362386   7.473925   
           INTC      120.133373  ...  551.163055   0.352735   6.555411   
...                         ...  ...         ...        ...        ...   
2025-02-28 MHK       111.989073  ...  535.244583   0.535644   5.935339   
           NCLH      108.488325  ...  573.581084   0.368484   5.951138   
           ON        106.078979  ...  585.018505   0.417526   7.623598   
           URI       128.688662  ...  582.147622   1.230024  41.351621   
           VST       128.874575  ...  555.572254   0.508578  45.833511   

                            %D       MACD  Histogram    Signal           OBV  \
Date       Asset                                                               
2025-01-01 LINK-USD  31.031936   6.163310  -3.163203  9.326513  9.416011e+11   
2025-01-02 ALB        9.225211   4.107428  -4.275821  8.383249  9.439234e+11   
           BLDR      11.383345  -2.379697  -6.604663  4.224966  9.434023e+11   
           CZR        9.990089  -9.690024 -11.131992  1.441968  9.433976e+11   
           INTC       8.007922 -16.293841 -14.188647 -2.105194  9.433512e+11   
...                        ...        ...        ...       ...           ...   
2025-02-28 MHK        5.101166 -12.748004  -8.731379 -4.016625  1.250758e+12   
           NCLH       4.459002 -17.470936 -10.763449 -6.707487  1.250734e+12   
           ON         6.503358 -19.031284  -9.859038 -9.172246  1.250747e+12   
           URI       19.392743  33.108894  34.274395 -1.165501  1.258292e+12   
           VST       32.129381  27.463567  22.903254  4.560313  1.258279e+12   

                     EMASignal  TotalSignal  
Date       Asset                             

In [165]:
def calculate_fee(amount):
    return max(0.00025 * amount, 0.50)

In [176]:
def auto_trade(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(['date', 'asset'])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {'units': 0, 'entry_price': None} for asset in assets}
    cash = config['starting_cash']
    trade_log = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )

        # Calculate thresholds
        cash_floor = config['cash_floor_pct'] * total_value
        #cash_ceiling = config['cash_ceiling_pct'] * total_value
    
        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row['Close']
            signal = row['TotalSignal']
            strategy_map.loc[asset, 'type']
            asset_type = strategy_map.loc[asset, 'type']
            #asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, 'weight']
            #ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]['units'] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config['buy_pct'] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            if portfolio[asset]['units'] > 0:
                entry_price = portfolio[asset]['entry_price']
                profit_pct = (current_price - entry_price) / entry_price
                if profit_pct >= config['tp_pct'] or profit_pct <= -config['sl_pct']: #IMPLEMENT DYNAMIC TP AND SL
                    if asset_type == 'Crypto':
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]['units'] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]['units'] * current_price)
                        proceeds = portfolio[asset]['units'] * current_price - fee
                    cash += proceeds
                    trade_log.append({
                        'asset': asset, 'date': date,
                        'action': 'SELL_TP' if profit_pct >= config['tp_pct'] else 'SELL_SL', 
                        'price': current_price, 'units': portfolio[asset]['units']
                    })
                    portfolio[asset] = {'units': 0, 'entry_price': None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]['units'] > 0:
                if asset_type == 'Crypto':
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]['units'] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]['units'] * current_price)
                    proceeds = portfolio[asset]['units'] * current_price - fee
                cash += proceeds
                trade_log.append({
                    'asset': asset, 'date': date, 'action': 'SELL_SHORT',
                    'price': current_price, 'units': portfolio[asset]['units']
                })
                portfolio[asset] = {'units': 0, 'entry_price': None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get('Expected Return', None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

            # Check if enough cash remains after purchase
                if buy_amount > 0 and (cash - buy_amount) >= cash_floor:
                    if asset_type == 'Crypto':
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price
                
                    old_units = portfolio[asset]['units']
                    old_price = portfolio[asset]['entry_price']

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price) / total_units
                        if old_units > 0 else current_price
                    ) #update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]['units'] = total_units
                    portfolio[asset]['entry_price'] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append({
                        'asset': asset, 'date': date, 'action': 'BUY_ADD' if old_units > 0 else 'BUY_NEW',
                        'price': current_price,'units': new_units
                    })
                
    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]['units'] * df.loc[(dates[-1], asset), 'Close']
        for asset in assets if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config['starting_cash']

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Export to CSV
        trade_df.to_csv("trade_log.csv", index=False)
        print("Trade log exported to 'trade_log.csv'")

    else:
        print("\n(No trades were executed during the period.)")

    return trade_df, final_value            

In [177]:
auto_trade(df_with_signals, strategy_map, config)

Index(['LINK-USD', 'ALB', 'BLDR', 'CZR', 'INTC', 'MHK', 'NCLH', 'ON', 'URI',
       'VST'],
      dtype='object', name='Asset')

Final Portfolio Value: 1002201.38
Total Profit: 2201.38
Number of Trades: 15
Trade log exported to 'trade_log.csv'


(       asset        date   action       price       units
 0   LINK-USD  2025-01-17  BUY_NEW   25.112717   34.122695
 1   LINK-USD  2025-01-28  SELL_SL   22.615997   34.122695
 2       NCLH  2025-01-31  BUY_NEW   28.350000   56.201281
 3         ON  2025-02-03  BUY_NEW   50.259998   70.205935
 4       NCLH  2025-02-07  BUY_ADD   27.330000   58.299435
 5        VST  2025-02-07  BUY_NEW  166.612076    1.599346
 6         ON  2025-02-10  SELL_SL   47.040001   70.205935
 7       NCLH  2025-02-11  SELL_SL   26.190001  114.500716
 8       INTC  2025-02-12  BUY_NEW   22.480000  354.457424
 9       INTC  2025-02-13  BUY_ADD   24.129999  330.411834
 10      INTC  2025-02-18  SELL_TP   27.389999  684.869258
 11       VST  2025-02-21  SELL_SL  150.180481    1.599346
 12      NCLH  2025-02-27  BUY_NEW   23.719999   67.330305
 13       VST  2025-02-27  BUY_NEW  129.785721    2.057998
 14       ALB  2025-02-28  BUY_NEW   76.599098   18.513121,
 1002201.3834808478)

## Previous functions 

In [ ]:
def load_data_and_apply_strategies(price_path, strategy_map_path):
    # 1. Load price data
    df_prices = pd.read_csv(price_path, parse_dates=['date'])

    # 2. Load asset-strategy mapping
    strat_df = pd.read_csv(strategy_map_path)

    # 3. Prepare DataFrame (MultiIndex)
    df_prices = df_prices.set_index(['date', 'asset']).sort_index()

    # 4. Dynamically load and apply strategies
    df_list = []
    strategy_instances = {}

    for _, row in strat_df.iterrows():
        asset = row['asset']
        strat_class_name = row['strategy']
        module_name = strat_class_name.lower()  # assumes strategy06.py for Strategy06 -- can be adjusted later based on our new naming

        # Load strategy class dynamically
        strategy_module = importlib.import_module(f'strategies.{module_name}')
        strategy_class = getattr(strategy_module, strat_class_name)
        strategy_instance = strategy_class()
        strategy_instances[asset] = strategy_instance

        # Apply strategy to that asset's data
        asset_df = df_prices.xs(asset, level='asset').copy()
        asset_df = strategy_instance.generate_signals(asset_df)
        asset_df['asset'] = asset
        df_list.append(asset_df.reset_index())

    # 5. Combine all assets back into one MultiIndex DataFrame
    full_df = pd.concat(df_list).set_index(['date', 'asset']).sort_index()

    return full_df, strat_df, strategy_instances

In [ ]:
#Gold
# ---------- GOLD Allocation ---------- only when cash reserve is higher than cash ceiling
        #Limitation: not converting gold to cash real-time if need to buy assets - missing opportunities
        if 'GOLD' in assets and date >= dates[15] and cash > cash_ceiling and (date, 'GOLD') in df.index:
            gold_row = df.loc[(date, 'GOLD')]
            gold_price = gold_row['Close']
            gold_signal = gold_row['TotalSignal']

            # Check TP/SL for gold
            if portfolio['GOLD']['units'] > 0:
                entry_price = portfolio['GOLD']['entry_price']
                profit_pct = (gold_price - entry_price) / entry_price
                if profit_pct >= config['tp_pct'] or profit_pct <= -config['sl_pct']:
                    proceeds = portfolio['GOLD']['units'] * gold_price * (1 - config['fee'])
                    cash += proceeds
                    trade_log.append({
                        'asset': 'GOLD',
                        'date': date,
                        'action': 'SELL_TP/SL_GOLD',
                        'price': gold_price,
                        'units': portfolio['GOLD']['units']
                    })
                    portfolio['GOLD'] = {'units': 0, 'entry_price': None}

            # SELL if signal is SHORT
            if gold_signal == 1 and portfolio['GOLD']['units'] > 0:
                proceeds = portfolio['GOLD']['units'] * gold_price * (1 - config['fee'])
                cash += proceeds
                trade_log.append({
                    'asset': 'GOLD',
                    'date': date,
                    'action': 'SELL_GOLD',
                    'price': gold_price,
                    'units': portfolio['GOLD']['units']
                })
                portfolio['GOLD'] = {'units': 0, 'entry_price': None}

            # BUY gold if LONG signal and cash > ceiling
            elif gold_signal == 2:
                excess_cash = cash - cash_ceiling
                invest_amount = min(excess_cash, 0.10 * total_value) #CAN CHANGE THE 0.10

                if invest_amount > 0 and (cash - invest_amount) >= cash_floor:
                    new_units = (invest_amount * (1 - config['fee'])) / gold_price
                    old_units = portfolio['GOLD']['units']
                    old_price = portfolio['GOLD']['entry_price']

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * gold_price) / total_units
                        if old_units > 0 else gold_price
                    )

                    portfolio['GOLD']['units'] = total_units
                    portfolio['GOLD']['entry_price'] = new_avg_price
                    cash -= invest_amount

                    trade_log.append({
                        'asset': 'GOLD',
                        'date': date,
                        'action': 'BUY_GOLD',
                        'price': gold_price,
                        'units': new_units
                    })